File: AmadeusTravelAPI-Flights.ipynb  
Name: Corinne Medeiros  
Date: 02/29/20    
Description: Program pulls flight data from the Amadeus Travel API, stores it in a Pandas dataframe, cleans up headers and columns, and saves the dataframe to a CSV file.  

In [133]:
import pandas as pd
import requests
import json

## Exploring Flight Options with Amadeus Travel API  

Visit <https://developers.amadeus.com/get-started/get-started-with-self-service-apis-335> to get your 'API Key' and 'API Secret' credentials before running this code.  

In [134]:
from amadeus import Client, ResponseError

# Initializing Amadeus library using your credentials
amadeus = Client(
    client_id='API KEY',
    client_secret='API SECRET'
)

![Airplane taking off](https://developers.amadeus.com/PAS-EAS/api/v1/cms-gateway//sites/default/files/styles/image_homepage_crop/public/2020-02/Airplane%20takeoff%20at%20Rio%20de%20Janeiro.jpg?h=8dab17aa&itok=nEPMsFAH)

(photo courtesty of Amadeus Travel API: <https://developers.amadeus.com/PAS-EAS/api/v1/cms-gateway//sites/default/files/styles/image_homepage_crop/public/2020-02/Airplane%20takeoff%20at%20Rio%20de%20Janeiro.jpg?h=8dab17aa&itok=nEPMsFAH>)

## Scenario

I'm interested in visiting Los Angeles this summer on July 9th. I'm pulling data from the Amadeus Travel API (<https://developers.amadeus.com/>) to see what my flight options are. 

## Initial Data Pull

In [141]:
try:
    '''
    What are the flight options for flying to Los Angeles from San Francisco on July 9th, 2020?
    '''
    response = amadeus.shopping.flight_offers.get(origin='SFO', destination='LAX', departureDate='2020-07-09')

    print(response.data)

except ResponseError as error:
    print(error)

[{'type': 'flight-offer', 'id': '1583015327667-1547356594', 'offerItems': [{'services': [{'segments': [{'flightSegment': {'departure': {'iataCode': 'SFO', 'terminal': '2', 'at': '2020-07-09T10:40:00-07:00'}, 'arrival': {'iataCode': 'LAX', 'terminal': '6', 'at': '2020-07-09T12:18:00-07:00'}, 'carrierCode': 'AS', 'number': '3432', 'aircraft': {'code': 'E75'}, 'operating': {'number': '3432'}, 'duration': '0DT1H38M'}, 'pricingDetailPerAdult': {'travelClass': 'ECONOMY', 'fareClass': 'X', 'availability': 7, 'fareBasis': 'TH2OAVBN'}}]}], 'price': {'total': '63.85', 'totalTaxes': '16.85'}, 'pricePerAdult': {'total': '63.85', 'totalTaxes': '16.85'}}]}, {'type': 'flight-offer', 'id': '1583015327667--702364065', 'offerItems': [{'services': [{'segments': [{'flightSegment': {'departure': {'iataCode': 'SFO', 'terminal': '2', 'at': '2020-07-09T16:00:00-07:00'}, 'arrival': {'iataCode': 'LAS', 'terminal': '3', 'at': '2020-07-09T17:37:00-07:00'}, 'carrierCode': 'AS', 'number': '3386', 'aircraft': {'code

## Visualizing Data Structure

Since the above data response is very nested and hard to visualize, I pasted the response into a JSON formatter (<https://jsonformatter.curiousconcept.com/>) to see the structure and grab the specific data I wanted.

Here is an example of one formatted entry to visualize the data structure:

<img src="JSON_formatter.png">

## Pulling and Storing Specific Data

Now, with a better understanding of the structure, I can pull specific variables using a key or an index.

In [135]:
# Creating an empty Pandas dataframe to store flight data
flightsdf = pd.DataFrame(columns=['flight_id'])

try:
    '''
    What are the flight options for flying to Los Angeles from San Francisco on July 9th, 2020?
    '''
    response = amadeus.shopping.flight_offers.get(origin='SFO', destination='LAX', departureDate='2020-07-09')

    # Looping through response, adding each variable to Pandas dataframe
    for r in response.data:

        flightsdf = flightsdf.append({'flight_id': r['id'], 
                                      'total':  r['offerItems'][0]['price']['total'],
                                     'totalTaxes': r['offerItems'][0]['price']['totalTaxes'], 
                                      'departure_port': r['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['departure']['iataCode'],
                                      'arrival_port': r['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['arrival']['iataCode'],
                                      'd_time': r['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['departure']['at'],
                                      'a_time': r['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['arrival']['at'],
                                      'd_fl_num': r['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['number'],
                                      'carrier': r['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['carrierCode'],
                                      'duration': r['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['duration'],
                                      'class': r['offerItems'][0]['services'][0]['segments'][0]['pricingDetailPerAdult']['travelClass'],
                                      'availability': r['offerItems'][0]['services'][0]['segments'][0]['pricingDetailPerAdult']['availability'],
                                      'aircraft_code': r['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['aircraft']['code'],
                                      'fare_class': r['offerItems'][0]['services'][0]['segments'][0]['pricingDetailPerAdult']['fareClass']
                                     
                                     },
                                     ignore_index=True)

except ResponseError as error:
    print(error)

    
print(flightsdf)

                    flight_id                     a_time aircraft_code  \
0   1583014646520--1088268279  2020-07-09T19:15:00-07:00           E75   
1    1583014646520-1686947287  2020-07-09T21:45:00-07:00           319   
2    1583014646520--161207687  2020-07-09T11:35:00-07:00           73J   
3   1583014646520--1143547020  2020-07-09T08:38:00-07:00           E75   
4    1583014646520-2125040166  2020-07-09T23:48:00-07:00           E75   
5     1583014646520-101227269  2020-07-09T19:30:00-07:00           CRJ   
6   1583014646520--1666249787  2020-07-09T13:14:00-07:00           E75   
7     1583014646520-795525582  2020-07-09T15:36:00-07:00           E75   
8    1583014646520-2070373450  2020-07-09T09:15:00-07:00           CRJ   
9     1583014646520-108207099  2020-07-09T18:44:00-07:00           739   
10   1583014646520-1499058599  2020-07-09T15:38:00-07:00           738   
11   1583014646520-1021995922  2020-07-10T00:24:00-07:00           319   
12    1583014646520-389555624  2020-07

In [136]:
# Summary of dataframe
print(flightsdf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
flight_id         50 non-null object
a_time            50 non-null object
aircraft_code     50 non-null object
arrival_port      50 non-null object
availability      50 non-null float64
carrier           50 non-null object
class             50 non-null object
d_fl_num          50 non-null object
d_time            50 non-null object
departure_port    50 non-null object
duration          50 non-null object
fare_class        50 non-null object
total             50 non-null object
totalTaxes        50 non-null object
dtypes: float64(1), object(13)
memory usage: 5.5+ KB
None


## Dataframe Clean Up

Our arrival time and departure time columns include a date and a time. It would make more sense to have this info split into two variables in separate columns.

I'll use string methods and Pandas to split them at "T" and then store them in separate columns.

In [137]:
# New arrival time data frame with split value columns 
a_time_df = flightsdf["a_time"].str.split("T", n = 1, expand = True)  

# Making separate date column from new data frame
flightsdf["a_date"]= a_time_df[0] 
  
# Making separate time column from new data frame
flightsdf["a_time_new"]= a_time_df[1]
  
# Dropping old time column 
flightsdf.drop(columns =["a_time"], inplace = True) 
  
# Previewing cleaned data 
print(flightsdf.head())

                   flight_id aircraft_code arrival_port  availability carrier  \
0  1583014646520--1088268279           E75          LAX           7.0      AS   
1   1583014646520-1686947287           319          LAX           9.0      DL   
2   1583014646520--161207687           73J          SEA           7.0      AS   
3  1583014646520--1143547020           E75          LAX           7.0      AS   
4   1583014646520-2125040166           E75          LAX           7.0      AS   

     class d_fl_num                     d_time departure_port  duration  \
0  ECONOMY     3486  2020-07-09T17:40:00-07:00            SFO  0DT1H35M   
1  ECONOMY     1898  2020-07-09T20:09:00-07:00            SFO  0DT1H36M   
2  ECONOMY      379  2020-07-09T09:15:00-07:00            SFO  0DT2H20M   
3  ECONOMY     3320  2020-07-09T07:00:00-07:00            SFO  0DT1H38M   
4  ECONOMY     3496  2020-07-09T22:15:00-07:00            SFO  0DT1H33M   

  fare_class   total totalTaxes      a_date      a_time_new  


In [138]:
# New departure time data frame with split value columns 
d_time_df = flightsdf["d_time"].str.split("T", n = 1, expand = True)

# Making separate date column from new data frames
flightsdf["d_date"]= d_time_df[0]

# Making separate time column from new data frames
flightsdf["d_time_new"]= d_time_df[1]

# Dropping old time column 
flightsdf.drop(columns =["d_time"], inplace = True)

# Previewing cleaned data 
print(flightsdf.head())

                   flight_id aircraft_code arrival_port  availability carrier  \
0  1583014646520--1088268279           E75          LAX           7.0      AS   
1   1583014646520-1686947287           319          LAX           9.0      DL   
2   1583014646520--161207687           73J          SEA           7.0      AS   
3  1583014646520--1143547020           E75          LAX           7.0      AS   
4   1583014646520-2125040166           E75          LAX           7.0      AS   

     class d_fl_num departure_port  duration fare_class   total totalTaxes  \
0  ECONOMY     3486            SFO  0DT1H35M          X   80.00      18.00   
1  ECONOMY     1898            SFO  0DT1H36M          E   77.81      17.81   
2  ECONOMY      379            SFO  0DT2H20M          X  135.46      29.46   
3  ECONOMY     3320            SFO  0DT1H38M          X   63.85      16.85   
4  ECONOMY     3496            SFO  0DT1H33M          X   73.49      17.49   

       a_date      a_time_new      d_date   

Now that I have them successfully separated, I'll clean up those column names.

In [139]:
flightsdf=flightsdf.rename(columns = {'a_time_new':'arr_time', 'd_time_new':'dep_time', 'a_date':'arr_date', 'd_date':'dep_date'})

print(flightsdf.head())

                   flight_id aircraft_code arrival_port  availability carrier  \
0  1583014646520--1088268279           E75          LAX           7.0      AS   
1   1583014646520-1686947287           319          LAX           9.0      DL   
2   1583014646520--161207687           73J          SEA           7.0      AS   
3  1583014646520--1143547020           E75          LAX           7.0      AS   
4   1583014646520-2125040166           E75          LAX           7.0      AS   

     class d_fl_num departure_port  duration fare_class   total totalTaxes  \
0  ECONOMY     3486            SFO  0DT1H35M          X   80.00      18.00   
1  ECONOMY     1898            SFO  0DT1H36M          E   77.81      17.81   
2  ECONOMY      379            SFO  0DT2H20M          X  135.46      29.46   
3  ECONOMY     3320            SFO  0DT1H38M          X   63.85      16.85   
4  ECONOMY     3496            SFO  0DT1H33M          X   73.49      17.49   

     arr_date        arr_time    dep_date   

The duration column could also use some cleaning. Since none of the flights are longer than a day, I'll strip the data to only include hours and minutes.

In [140]:
# New duration data frame with stripped values 
dur_df = flightsdf["duration"].str.split("T", n = 1, expand = True)

# Making new duration column from new data frame
flightsdf["duration_new"]= dur_df[1]

# Dropping old duration column 
flightsdf.drop(columns =["duration"], inplace = True)

# Renaming cleaned duration column
flightsdf=flightsdf.rename(columns = {'duration_new':'duration'})

# Previewing cleaned data 
print(flightsdf.head())

                   flight_id aircraft_code arrival_port  availability carrier  \
0  1583014646520--1088268279           E75          LAX           7.0      AS   
1   1583014646520-1686947287           319          LAX           9.0      DL   
2   1583014646520--161207687           73J          SEA           7.0      AS   
3  1583014646520--1143547020           E75          LAX           7.0      AS   
4   1583014646520-2125040166           E75          LAX           7.0      AS   

     class d_fl_num departure_port fare_class   total totalTaxes    arr_date  \
0  ECONOMY     3486            SFO          X   80.00      18.00  2020-07-09   
1  ECONOMY     1898            SFO          E   77.81      17.81  2020-07-09   
2  ECONOMY      379            SFO          X  135.46      29.46  2020-07-09   
3  ECONOMY     3320            SFO          X   63.85      16.85  2020-07-09   
4  ECONOMY     3496            SFO          X   73.49      17.49  2020-07-09   

         arr_time    dep_date   

In [129]:
# Writing cleaned dataframe to CSV file
flightsdf.to_csv('flights_sfo_lax_07092020.csv')

With the CSV file successfully created, I feel great!

![Female Traveler.](https://developers.amadeus.com/PAS-EAS/api/v1/cms-gateway//sites/default/files/styles/blog_list_image/public/2019-12/Back%20view%20of%20redheaded%20woman%20sitting%20on%20a%20rockt%20looking%20at%20distance_0.jpg?itok=Apc9D7Jb)

(photo courtesy of Amadeus Travel API: <https://developers.amadeus.com/PAS-EAS/api/v1/cms-gateway//sites/default/files/styles/blog_list_image/public/2019-12/Back%20view%20of%20redheaded%20woman%20sitting%20on%20a%20rockt%20looking%20at%20distance_0.jpg?itok=Apc9D7Jb>)